In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
import yaml
from datetime import datetime
import torch

# 1: Choose innerloop and outerloop

In [8]:
from Innerloops.Innerloop_bg_TIF_IE_6pPol import *
innerloop_name = 'Innerloop_bg_TIF_IE_6pPol'
####  Example options:
# Stability task, only I-to-E plasticity, small polynomial rule: 'Innerloop_bg_TIF_IE_6pPol'
# Familiarity detection task, only I-to-E plasticity, small polynomial rule: 'Innerloop_FamDet_Lbasic_TIF_IE_6pPol'
# Stability task, only I-to-E plasticity, MLP rule:  'Innerloop_bg_CVAIF_IE_T4wvceciMLP'
# Stability task, only I-to-E plasticity, big polynomial rule: 'Innerloop_bg_CVAIF_IE_T4wvceciPol'
 
from Outerloops.Outerloop_CMA_ES import Outerloop_CMA_ES
outerloop_name = "Outerloop_CMA_ES"

# 2: Innerloop parameters

#### Stability task, only I-to-E plasticity, small polynomial rule

In [9]:
if innerloop_name == 'Innerloop_bg_TIF_IE_6pPol':
    
    #load most task parameters from yaml
    task_name = "bg_TIF_IE_6pPol"
    with open("Innerloops/tasks_configs/%s.yaml" % task_name, "r") as f:
        innerloop_params = yaml.load(f, Loader=yaml.Loader)

    #initial plasticity rule
    A_chosen = [np.log(0.01), np.log(0.01), 0, 0, 0, 0]

#### Familiarity detection task, only I-to-E plasticity, small polynomial rule

In [ ]:
if innerloop_name == 'Innerloop_FamDet_Lbasic_TIF_IE_6pPol':
    
    #load most task parameters from yaml
    task_name = "FamDet_Lbasic_TIF_IE_6pPol"
    with open("Innerloops/tasks_configs/%s.yaml" % task_name, "r") as f:
        innerloop_params = yaml.load(f, Loader=yaml.Loader)

    #initial plasticity rule
    A_chosen = [np.log(0.01), np.log(0.01), 0, 0, 0, 0]

#### Stability task, only I-to-E plasticity, MLP rule

In [ ]:
if innerloop_name == 'Innerloop_bg_CVAIF_IE_T4wvceciMLP':
    task_name = "bg_CVAIF_IE_T4wvceciMLP"
    with open("Innerloops/tasks_configs/%s.yaml" % task_name, "r") as f:
        innerloop_params = yaml.load(f, Loader=yaml.Loader)
    
    # Choose initial plasticity rule
    net = torch.load("Saves/test_model")
    nh1 = net.l1.weight.shape[0]
    nh2 = net.l2.weight.shape[0]
    array_rule = convert_wmats_to_vector_for_auryn(eta=0.1,
                                  tau_pre1=0.01,
                                  tau_pre2=0.01,
                                  tau_post1=0.1,
                                  tau_post2=0.1,
                                  Wh1=net.l1.weight,
                                  Wh2=net.l2.weight,
                                  Wpre=torch.concat((net.l3.weight, net.l3.bias.reshape(-1,1)), dim=1),
                                  Wpost=torch.concat((net.l3.weight, net.l3.bias.reshape(-1,1)), dim=1))
    rule_constant_part =  array_rule[1:-(nh2+1)*2]
    innerloop_params['nh1']=nh1; innerloop_params['nh2']=nh2
    innerloop_params['rule_constant_part']=rule_constant_part
    
    #only learn eta and the final layer weights
    A_chosen = np.concatenate(([array_rule[0]], array_rule[-(nh2+1)*2:]))
    print(A_chosen.shape, A_chosen)

#### Stability task, only I-to-E plasticity, big polynomial rule

In [ ]:
if innerloop_name == 'Innerloop_bg_CVAIF_IE_T4wvceciPol':
    task_name = "bg_CVAIF_IE_T4wvceciPol"
    with open("Innerloops/tasks_configs/%s.yaml" % task_name, "r") as f:
        innerloop_params = yaml.load(f, Loader=yaml.Loader)
    
    # random initial rule (CMA does the jitter anyway)
    A_chosen = np.zeros(21)
    
    print(A_chosen.shape, A_chosen)

# 3: Outerloop parameters

In [10]:
if outerloop_name == "Outerloop_CMA_ES":
    # to play with
    n_meta_it = 1
    reg = 0
    sigma = 0.001
    lambd = 10 #12 options "auto" or int

    outerloop_params = dict([('A', A_chosen),  ("sigma", sigma), ("reg", reg), ("lambd", lambd), ('inner_loop', innerloop_name)])

# 4: Meta-learning

In [11]:
innerloop_params

{'auryn_sim_dir': 'Innerloops/cpp_innerloops/',
 'hardware': 'local',
 'Nd': 10,
 'Lr': 6,
 'NE': 8000,
 'NI': 2000,
 'wee': 0.03,
 'wei': 0.03,
 'wie_low': 0.0,
 'wie_high': 0.3,
 'wii': 0.3,
 'eta': 0.01,
 'wmax_low': 2,
 'wmax_high': 3,
 'sparseness': 0.02,
 'length_training_phase': 30,
 'length_scoring_phase': 20,
 'length_scoring_window': 5,
 'target': 10,
 'N_inputs': 5000,
 'sparseness_poisson': 0.05,
 'w_poisson': 0.1,
 'poisson_rate_low': 1,
 'poisson_rate_high': 5,
 'min_rate_checker': 0.1,
 'max_rate_checker': 100,
 'tau_checker': 1,
 'workdir': 'Innerloops/cpp_innerloops/sim_workdir/'}

In [16]:
print("optimising", innerloop_name, "with", outerloop_name)
start = time.time()
outerloop = eval(outerloop_name + "(outerloop_params, innerloop_params)")
print("Initialisation took " + str(np.round(time.time()-start,2)) + "s")

optimising Innerloop_bg_TIF_IE_6pPol with Outerloop_CMA_ES
Initialisation took 0.0s


In [ ]:
start = time.time()
outerloop.run(n_meta_it) 
print("Execution took " + str(np.round(time.time()-start,2)) + "s")
outerloop.plot()
print(outerloop)
print(outerloop.innerloop)

In [ ]:
today = datetime.now()
d2 = today.strftime("%y%m%d%H%M")
np.savez(d2 + "_" + task_name + "_CMA", loss_hist=outerloop.loss_hist, rule_hist=outerloop.rule_hist, C_hist=outerloop.C_hist)